In [1]:
import sys
import os

# Add the actual path where 'data_sources' exists
dagster_path = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Goes from 'dags/' to 'dagster/'
if dagster_path not in sys.path:
    sys.path.insert(0, dagster_path)

print(f"✅ 'dagster' path added to sys.path: {dagster_path}")

✅ 'dagster' path added to sys.path: c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\src\dagster


In [11]:
# ✅ Cell 2: Import Dagster and the asset
from dagster import build_op_context
from data_sources.external_db_ingest import external_data
from data_sources.csv_mapping_ingest import load_csv_data
from data_preprocessing.transaction_clean_merge import run_full_preprocessing

# Define DB config
db_config = {
    "database": "admin_aim_masterfile",
    "host": "162.241.4.11",
    "password": "7x9r7f@2W",  # Replace with actual password
    "port": 3306,
    "queries": [
        """
        SELECT data_id, branch, pos, transdate, ite_code, quantity, dep_code, date, time, type, delivery
        FROM admin_aim_masterfile.rd5000
        WHERE date > '2022-01-01' AND branch in ('OLA', 'BRLN');
        """,
        """
        SELECT data_id, incode, ite_desc, dep_code, unit_prc, branch, pos
        FROM admin_aim_masterfile.rd5500
        WHERE branch in ('OLA', 'BRLN');
        """,
        """
        SELECT data_id, dept_code, dept_name, branch, pos
        FROM admin_aim_masterfile.rd1800
        WHERE branch in ('OLA', 'BRLN');
        """
    ],
    "user": "admin_aim_sandbox"
}

csv_config = {
    "department_filepath": "../../../csv/department_category_mapping.csv",
    "item_filepath": "../../../csv/item_category_mapping.csv"
}

In [13]:
# Simulate Dagster execution contexts
db_context = build_op_context(config=db_config)
csv_context = build_op_context(config=csv_config)

# Run data loaders
external_outputs = external_data(db_context)
csv_outputs = load_csv_data(csv_context)

2025-04-20 17:09:29 +0800 - dagster - INFO - system - Connected to external MariaDB database.
c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\src\dagster\data_sources\external_db_ingest.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-04-20 17:14:04 +0800 - dagster - INFO - system - Query 0 executed; retrieved 835278 rows.
2025-04-20 17:14:06 +0800 - dagster - INFO - system - Query 1 executed; retrieved 7368 rows.
2025-04-20 17:14:06 +0800 - dagster - INFO - system - Query 2 executed; retrieved 129 rows.
2025-04-20 17:14:06 +0800 - dagster - INFO - system - Database connection closed.
2025-04-20 17:14:06 +0800 - dagster - INFO - system - Loading CSV files from ../../../csv/department_category_mapping.csv and ../../../csv/item_category_mapping.csv
2025-04-20 17:14:06 +0800 -

In [14]:
# Run full data cleaning and enrichment pipeline
final_df = run_full_preprocessing(external_outputs, csv_outputs)

print("✅ Final dataset shape:", final_df.shape)
final_df.head(10)  # Display top rows

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\venv\Lib\site-packages\sqlalchemy\pool\base.py", line 987, in _finalize_fairy
    fairy._reset(
  File "c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\venv\Lib\site-packages\sqlalchemy\pool\base.py", line 1433, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\venv\Lib\site-packages\sqlalchemy\engine\default.py", line 703, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: Cannot operate on a closed database.
Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\venv\Lib\site-packages\sqlalchemy\pool\base.py", line 987, in _finalize_fairy
    fairy._reset(
  File "c:\Users\RanneLMarquez\OneDrive - RAMCAR\Documents\GitHub\aim-biggs\venv\Lib\s

✅ Final dataset shape: (738312, 10)


,data_id,branch,quantity,date,time,type,dept_name_std,ite_desc_std,food_type,cat_flag
0,424923,OLA,1,2023-12-31,21:53:00,D,SALAD DELIGHTS,ROASTED SESAME DRESSING,condiments,NaN
1,424924,OLA,1,2023-12-31,21:53:00,D,SALAD DELIGHTS,NENA'S HARVEST,food,NaN
12,424935,OLA,1,2023-12-31,21:46:00,D,CLASSIC MEALS,PORK CORDON BLEU,food,NaN
13,424936,OLA,1,2023-12-31,21:46:00,D,CHICKEN MEALS,SPAGHETTI W CHICKEN,food,NaN
14,424937,OLA,1,2023-12-31,21:46:00,D,BEVERAGES,ICED TEA,beverage,NaN
15,424938,OLA,1,2023-12-31,21:45:00,D,NESPRESSO,BREWED COFFEE,beverage,NaN
16,424938,OLA,1,2023-12-31,21:45:00,D,NESPRESSO,NESPRESO COFFEE,beverage,NaN
17,424939,OLA,1,2023-12-31,21:45:00,D,SALAD DELIGHTS,CREAMY MACARONI SALAD,food,NaN
18,424940,OLA,1,2023-12-31,21:41:00,D,PASTA DISHES,SPAGHETTI,food,NaN
19,424941,OLA,1,2023-12-31,21:41:00,D,BURGERS AND SANDWICHES,CHEESY BURGER,food,NaN
